[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/elsamuel27/snapthon/blob/main/Snapthon.ipynb)

In [15]:
#@title # Paso 1: Rellena el formulario a continuación y ejecuta todas las celdas

mult = "Playlist" #@param ["Video", "Playlist"]
link = "https://youtube.com/playlist?list=PLnErfpyW0Y5LgjrwKwu3E2OE2TbqXqBvv" #@param {type: "string"}
ft = "Audio" #@param ["Video", "Audio"]
format = "mp3" #@param ["mp3", "mp4"] {allow-input: true}
route = "/content/Descargados"
purification = True #@param {type: "boolean"}
#@markdown La purificación es un proceso que hace que la música sea más propensa a ser compatible con coches, casets, etc, pero aumenta mucho el tiempo de descarga. Actívala marcando la casilla de arriba. No tiene ningún efecto al momento de descargar vídeo, por lo que se desactivará automáticamente
zip = True #@param {type: "boolean"}
#@markdown Si vas a descargar una playlist con bastantes vídeos, es recomendable que marques la casilla de arriba para recibir tu contenido en un archivo ZIP
if ft != "Audio" and purification:
    purification = False

Para ejecutar todas las celdas haz click en `Entorno de ejecución > Ejecutar todas`, o pulsa `Ctrl + F9`

# Paso 2: Espera. En breve tendrás tus descargas listas

In [2]:
#@title Actualizaciones

!pip install -U pip
!sudo apt update -y

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 23.9 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 22.0.4
    Uninstalling pip-22.0.4:
      Successfully uninstalled pip-22.0.4
Hit:1 http://archive.ubuntu.com/ubuntu focal InRelease
Get:2 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Get:5 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:7 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease [18.1 kB]
Hit:10

In [7]:
#@title Instalaciones
with open("requirements.txt", "a") as r:
  r.write("pytube\n")
  if purification:
    r.write("pydub\n")
  r.close()

!pip install -r requirements.txt
!sudo apt install zip -y

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 2.7 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree       
Reading state information... Done
zip is already the newest version (3.0-11build1).
0 upgraded, 0 newly installed, 0 to remove and 23 not upgraded.


In [8]:
#@title Importaciones

import os
from pytube import YouTube
from pytube import Playlist
try:
    from google.colab import files
except:
    print("Uso fuera de Google Colab activado")
if purification:
    from pydub import AudioSegment

Una desventaja de que los nombres de los archivos descargados sean los títulos de los vídeos, es que pueden contener caracteres incompatibles con el sistema de archivos. Una solución sencilla que se me ha ocurrido es la siguiente: 

- Creamos una lista con todos los caracteres no permitidos por Windows (Pues Linux admite unos cuántos más y para curar en salud mejor tener en cuenta más caracteres)
- En vez de usar el título directamente, lo almacenamos en otra variable, y acto seguido usamos el método `.replace()` de las cadenas de caracteres para reemplazar los caracteres no permitidos por nada, ergo, borrarlos

In [9]:
#@title Descargas en el entorno

characters = ["\\", "/", ":", "*", "?", "\"", "<", ">", "|"]

if mult == "Video":
    yt = YouTube(link)
    title = yt.title
    for c in characters:
      title = title.replace(c, "")
    file = f"{title}.{format}"

    if ft == "Video":
      yd = yt.streams.get_highest_resolution()
      yd.download(route, file)
      print("¡Vídeo descargado! Hasta luego :D")
    elif ft == "Audio":
      yd = yt.streams.get_audio_only()
      yd.download(route, file)
      print("¡Vídeo descargado! Hasta luego :D")
elif mult == "Playlist":
    pl = Playlist(link)
    if ft == "Video":
      for video in pl.videos:
        title = video.title
        for c in characters:
          title = title.replace(c, "")
        file = f"{title}.{format}"
        try:
          yd = video.streams.get_highest_resolution()
          yd.download(route, file)
          print("+1 vídeo en la carpeta")
        except:
          print("No se puede acceder a los datos de este vídeo. Saltamos al siguiente")
    elif ft == "Audio":
      for video in pl.videos:
        title = video.title
        for c in characters:
          title = title.replace(c, "")
        try:
          yd = video.streams.get_audio_only()
          yd.download(route, f"{title}.{format}")
          print("+1 vídeo en la carpeta")
        except:
          print("No se puede acceder a los datos de este vídeo. Saltamos al siguiente")

+1 vídeo en la carpeta
+1 vídeo en la carpeta


In [12]:
#@title Purificación (Si se ha activado)
if purification:
    input_folder = "/content/Descargados"
    output_folder = "/content/PDescargados"
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for file_name in os.listdir(input_folder):
        input_file_path = os.path.join(input_folder, file_name)
        output_file_path = os.path.join(output_folder, file_name.rsplit('.', 1)[0] + '.mp3')
        sound = AudioSegment.from_file(input_file_path)
        sound.export(output_file_path, format='mp3')
    os.system("mv ./PDescargados ./bySnapthon")
    os.system("rm -rf ./Descargados")
else:
  os.system("mv ./Descargados ./bySnapthon")

In [17]:
#@title Descarga

if zip:
  os.system("zip -r /content/bySnapthon.zip ./bySnapthon")
  try:
    files.download("/content/bySnapthon.zip")
  except:
    print("Uso fuera de Google Colab detectado.")
else:
  try:
    for video in os.listdir("/content/bySnapthon"):
      files.download(f"/content/bySnapthon/{video}")
  except:
    print("Uso fuera de Google Colab detectado")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Recomendada la primera opción para listas con muchos vídeos, la segunda para unos pocos o directamente uno

# Paso 3: Disfruta el contenido descargado